# Creating a Logistic Regression Model to Predict Absenteeism

In [1]:
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Import the Relevant Libraries

In [2]:
import numpy as np
import pandas as pd

## Load the Data

In [3]:
data_preprocessed = pd.read_csv('Absenteeism_preprocessed.csv')

In [4]:
data_preprocessed.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.55,30,0,2,1,4.0
1,0,0,0,0,7,1,118,13,50,239.55,31,0,1,0,0.0
2,0,0,0,1,7,2,179,51,38,239.55,31,0,0,0,2.0
3,1,0,0,0,7,3,279,5,39,239.55,24,0,2,0,4.0
4,0,0,0,1,7,3,289,36,33,239.55,30,0,2,1,2.0


## Create the Targets

In this model, I choose median of *'Absenteeism Time in Hours'* as the threshold. Anyone who is absent for more than the median hours will be considered as *Excessively Absent (1)* otherwise *Moderately Absent (0)*.

The targets have been calssified in two categories, making it a logistic problem. 

In [5]:
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [6]:
targets = np.where(data_preprocessed['Absenteeism Time in Hours'] > data_preprocessed['Absenteeism Time in Hours'].median(), 1, 0)

In [7]:
targets

array([1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0,

In [8]:
data_preprocessed['Excessive Absenteeism'] = targets

In [9]:
data_preprocessed.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.55,30,0,2,1,4.0,1
1,0,0,0,0,7,1,118,13,50,239.55,31,0,1,0,0.0,0
2,0,0,0,1,7,2,179,51,38,239.55,31,0,0,0,2.0,0
3,1,0,0,0,7,3,279,5,39,239.55,24,0,2,0,4.0,1
4,0,0,0,1,7,3,289,36,33,239.55,30,0,2,1,2.0,0


### A Comment on the Targets
We check the preprocessed data is balanced or not, *i.e.*, whether there are as many 1(s) present in the data as there are 0(s). 

In [10]:
targets.sum() / targets.shape[0]

0.4014285714285714

There is a roughly 46:54 percent ratio between Number of Ones and Number of Zeroes, which is good enough to proceed for further processing.

In [11]:
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours', 'Day of the Week', 'Daily Work Load Average', 'Distance to Work'], axis=1)

We drop the *'Absenteeism Time in Hours'* as it is no more needed in any further preocessing.

Also, we drop the *'Day of the Week'*, *'Daily Work Load Average'* and *'Distance to Work'* columns. At the time of training the model, it was observed that these features do not affect the model much as their coefficient values were too small, approximately ~0.0003.

In [12]:
data_with_targets is data_preprocessed

False

In [13]:
data_with_targets.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,0,0,0,1,7,289,33,30,0,2,1,1
1,0,0,0,0,7,118,50,31,0,1,0,0
2,0,0,0,1,7,179,38,31,0,0,0,0
3,1,0,0,0,7,279,39,24,0,2,0,1
4,0,0,0,1,7,289,33,30,0,2,1,0


## Selecting the Inputs for the Regression

In [14]:
data_with_targets.shape

(700, 12)

In [15]:
data_with_targets.iloc[:, :-1]

,Reason 1,Reason 2,Reason 3,Reason 4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,0,0,0,1,7,289,33,30,0,2,1
1,0,0,0,0,7,118,50,31,0,1,0
2,0,0,0,1,7,179,38,31,0,0,0
3,1,0,0,0,7,279,39,24,0,2,0
4,0,0,0,1,7,289,33,30,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,5,179,40,22,1,2,0
696,1,0,0,0,5,225,28,24,0,1,2
697,1,0,0,0,5,330,28,25,1,0,0
698,0,0,0,1,5,235,32,25,1,0,0


In [16]:
unscaled_inputs = data_with_targets.iloc[:, :-1]

## Standardizing the Data

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class CustomScaler(BaseEstimator, TransformerMixin):
    def __init__(self, columns, copy=True, with_mean=True, with_std=True):
        self.columns = columns
        self.copy = copy
        self.with_mean = with_mean
        self.with_std = with_std
    
    def fit(self, X, y=None):
        self.scaler = StandardScaler(copy=self.copy, with_mean=self.with_mean, with_std=self.with_std)
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_not_scaled = X.loc[:, ~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [18]:
unscaled_inputs.columns.values

array(['Reason 1', 'Reason 2', 'Reason 3', 'Reason 4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

We omit *'Reason 1'*, *'Reason 2'*, *'Reason 3'*, *'Reason 4'* and *'Education'* columns for standardizing as these are the dummy categorical variables and standardizing them will lose the information they contain.

In [19]:
columns_to_omit = ['Reason 1', 'Reason 2', 'Reason 3', 'Reason 4', 'Education']

columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [20]:
absenteeism_scaler = CustomScaler(columns_to_scale)

In [21]:
absenteeism_scaler.fit(unscaled_inputs)

CustomScaler(columns=['Month Value', 'Transportation Expense', 'Age',
                      'Body Mass Index', 'Children', 'Pets'])

In [22]:
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

In [23]:
scaled_inputs

,Reason 1,Reason 2,Reason 3,Reason 4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.030796,1.005844,-0.536062,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.030796,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.030796,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
3,1,0,0,0,0.030796,0.854936,0.405184,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.030796,1.005844,-0.536062,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,-0.568019,-0.654143,0.562059,-1.114186,1,0.880469,-0.589690
696,1,0,0,0,-0.568019,0.040034,-1.320435,-0.643782,0,-0.019280,1.126663
697,1,0,0,0,-0.568019,1.624567,-1.320435,-0.408580,1,-0.919030,-0.589690
698,0,0,0,1,-0.568019,0.190942,-0.692937,-0.408580,1,-0.919030,-0.589690


In [24]:
scaled_inputs.shape

(700, 11)

## Split the Data into Train & Test and Shuffle

### Import the Relevant Module

In [25]:
from sklearn.model_selection import train_test_split

### Split

In [26]:
train_test_split(scaled_inputs, targets)

[     Reason 1  Reason 2  Reason 3  Reason 4  Month Value  \
 486         0         0         0         1     0.929019   
 131         0         0         0         1    -1.765648   
 677         1         0         0         0     0.629611   
 431         0         0         0         1     1.527833   
 512         0         0         0         1     0.929019   
 ..        ...       ...       ...       ...          ...   
 284         0         0         0         1     0.629611   
 44          0         0         0         1     0.629611   
 326         0         0         1         0     1.228426   
 409         0         0         0         1    -1.166834   
 634         1         0         0         0    -1.166834   
 
      Transportation Expense       Age  Body Mass Index  Education  Children  \
 486                1.036026  0.562059        -0.408580          0 -0.019280   
 131               -1.574681  0.091435         0.297027          0 -0.919030   
 677               -1.5746

In [27]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size = 0.8, random_state = 20)

In [28]:
print(x_train.shape, y_train.shape)

(560, 11) (560,)


In [29]:
print(x_test.shape, y_test.shape)

(140, 11) (140,)


## Logistic Regression with sklearn

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

### Train the model

In [31]:
reg = LogisticRegression()

In [32]:
reg.fit(x_train, y_train)

LogisticRegression()

In [33]:
print(f"The accuracy of our logistic regression model on train data is {(reg.score(x_train, y_train)) * 100 :.2f}%")

The accuracy of our logistic regression model on train data is 72.68%


### Manually Checking the Accuracy

In [34]:
model_outputs = reg.predict(x_train)

In [35]:
model_outputs

array([0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0,

In [36]:
y_train

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,

In [37]:
model_outputs == y_train

array([ True,  True, False, False,  True,  True,  True,  True,  True,
        True, False,  True, False, False,  True,  True,  True,  True,
       False,  True,  True,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False, False,  True,  True,  True,  True,  True, False,
        True,  True,  True, False,  True, False,  True,  True,  True,
        True, False,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
       False,  True,  True, False,  True,  True,  True, False,  True,
       False,  True, False,  True,  True, False, False, False, False,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True, False,  True, False,  True,  True, False,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
       False,  True, False,  True,  True, False,  True,  True, False,
        True,  True,

In [38]:
np.sum(model_outputs == y_train)

407

In [39]:
model_outputs.shape[0]

560

In [40]:
np.sum(model_outputs == y_train) / model_outputs.shape[0]

0.7267857142857143

### Finding the Intercept and Coefficients

In [41]:
reg.intercept_

array([-1.73732977])

In [42]:
reg.coef_

array([[ 2.24788414,  0.95481002,  2.55470232,  0.88319039,  0.05653064,
         0.52228161, -0.05355861,  0.16780546, -0.13098189,  0.30081469,
        -0.14199405]])

In [43]:
unscaled_inputs.columns.values

array(['Reason 1', 'Reason 2', 'Reason 3', 'Reason 4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [44]:
feature_name = unscaled_inputs.columns.values

In [45]:
summary_table = pd.DataFrame(columns = ['Feature Name'], data = feature_name)

summary_table['Coefficient'] = np.transpose(reg.coef_)

summary_table

,Feature Name,Coefficient
0,Reason 1,2.247884
1,Reason 2,0.954810
2,Reason 3,2.554702
3,Reason 4,0.883190
4,Month Value,0.056531
5,Transportation Expense,0.522282
6,Age,-0.053559
7,Body Mass Index,0.167805
8,Education,-0.130982
9,Children,0.300815


In [46]:
summary_table.index += 1

In [47]:
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]

In [48]:
summary_table = summary_table.sort_index()

In [49]:
summary_table

,Feature Name,Coefficient
0,Intercept,-1.737330
1,Reason 1,2.247884
2,Reason 2,0.954810
3,Reason 3,2.554702
4,Reason 4,0.883190
5,Month Value,0.056531
6,Transportation Expense,0.522282
7,Age,-0.053559
8,Body Mass Index,0.167805
9,Education,-0.130982


### Interpreting the Coefficients

In [50]:
summary_table['Odds Ratio'] = np.exp(summary_table.Coefficient)

In [51]:
summary_table

,Feature Name,Coefficient,Odds Ratio
0,Intercept,-1.737330,0.175990
1,Reason 1,2.247884,9.467682
2,Reason 2,0.954810,2.598177
3,Reason 3,2.554702,12.867469
4,Reason 4,0.883190,2.418604
5,Month Value,0.056531,1.058159
6,Transportation Expense,0.522282,1.685870
7,Age,-0.053559,0.947850
8,Body Mass Index,0.167805,1.182707
9,Education,-0.130982,0.877234


In [52]:
summary_table.sort_values('Odds Ratio', ascending = False)

,Feature Name,Coefficient,Odds Ratio
3,Reason 3,2.554702,12.867469
1,Reason 1,2.247884,9.467682
2,Reason 2,0.954810,2.598177
4,Reason 4,0.883190,2.418604
6,Transportation Expense,0.522282,1.685870
10,Children,0.300815,1.350959
8,Body Mass Index,0.167805,1.182707
5,Month Value,0.056531,1.058159
7,Age,-0.053559,0.947850
9,Education,-0.130982,0.877234


### Testing the Model

In [53]:
print(f"The accuracy of our model on test data is {(reg.score(x_test, y_test)) * 100 :.2f}%")

The accuracy of our model on test data is 69.29%


In [54]:
predicted_proba = reg.predict_proba(x_test)

In [55]:
predicted_proba

array([[0.76513567, 0.23486433],
       [0.60349236, 0.39650764],
       [0.55424118, 0.44575882],
       [0.78575503, 0.21424497],
       [0.17436896, 0.82563104],
       [0.48371357, 0.51628643],
       [0.42201855, 0.57798145],
       [0.21147517, 0.78852483],
       [0.73780254, 0.26219746],
       [0.77413734, 0.22586266],
       [0.51352938, 0.48647062],
       [0.27879528, 0.72120472],
       [0.14051543, 0.85948457],
       [0.71317762, 0.28682238],
       [0.33442354, 0.66557646],
       [0.58065566, 0.41934434],
       [0.50930019, 0.49069981],
       [0.53042076, 0.46957924],
       [0.40170817, 0.59829183],
       [0.14023971, 0.85976029],
       [0.72449629, 0.27550371],
       [0.78575503, 0.21424497],
       [0.61251857, 0.38748143],
       [0.59633291, 0.40366709],
       [0.35751721, 0.64248279],
       [0.74106361, 0.25893639],
       [0.54661959, 0.45338041],
       [0.85224029, 0.14775971],
       [0.3918149 , 0.6081851 ],
       [0.78575503, 0.21424497],
       [0.

In [56]:
predicted_proba.shape

(140, 2)

In [57]:
predicted_proba[:, 1]

array([0.23486433, 0.39650764, 0.44575882, 0.21424497, 0.82563104,
       0.51628643, 0.57798145, 0.78852483, 0.26219746, 0.22586266,
       0.48647062, 0.72120472, 0.85948457, 0.28682238, 0.66557646,
       0.41934434, 0.49069981, 0.46957924, 0.59829183, 0.85976029,
       0.27550371, 0.21424497, 0.38748143, 0.40366709, 0.64248279,
       0.25893639, 0.45338041, 0.14775971, 0.6081851 , 0.21424497,
       0.38042591, 0.56522928, 0.54578355, 0.49493034, 0.21424497,
       0.53538787, 0.26548489, 0.63524697, 0.43686345, 0.54742717,
       0.21140941, 0.38810372, 0.25249389, 0.33338612, 0.69576432,
       0.46229841, 0.55332735, 0.21999876, 0.27013718, 0.20860142,
       0.42070387, 0.24214541, 0.51628643, 0.25377954, 0.72003912,
       0.40463562, 0.84133012, 0.23730795, 0.24326464, 0.24639396,
       0.55552328, 0.52895403, 0.24662046, 0.66167891, 0.27239632,
       0.23183639, 0.07465924, 0.26879852, 0.65559702, 0.30086871,
       0.24615844, 0.29344687, 0.76202206, 0.42855409, 0.51851

## Saving the Model

In [58]:
import pickle

In [59]:
with open('absenteeism_model', 'wb') as file:
    pickle.dump(reg, file)

In [60]:
with open('absenteeism_scaler', 'wb') as file:
    pickle.dump(absenteeism_scaler, file)